In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        
        # First convolutional layer: input channels = 1, output channels = 16, kernel size = 3
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        
        # Second convolutional layer: input channels = 16, output channels = 32, kernel size = 3
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        
        self.fc = nn.Linear(1568, 10)

    def forward(self, x):
        # Apply first convolution, followed by ReLU and max pooling
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        
        # Apply second convolution, followed by ReLU and max pooling
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)

        # Pass through the fully connected layer
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.fc(x)
        return x

In [3]:
model = SimpleConvNet()
input_tensor = torch.randn(1, 1, 28, 28)  # Example input tensor of shape (batch_size, channels, height, width)
output = model(input_tensor)

In [4]:
def quantize(float_model:torch.nn.Module, 
             input_shape:tuple,
             quant_dir:str, 
             quant_mode:str, 
             device:torch.device):
    """
    :param float_model: float model with loaded weights
    :param input_shape: shape of input(CH,W,H)
    :param quant_dir: path to directory with quantized model components
    :param quant_mode: quant_mode in ['calib', 'test'] 
    :param data_loader: data_loader - for 'calib' must be batch_size == 1
    """
    # available in docker or after packaging 
    # vitis-AI-tools/..../pytorch../pytorch_nndct
    # and installing the package
    print("before import")
    from pytorch_nndct.apis import torch_quantizer, dump_xmodel
    # model to device
    print("Before device")
    model = float_model.to(device)

    # Force to merge BN with CONV for better quantization accuracy
    optimize = 1

    rand_in = torch.randn(input_shape)
    print("get qunatizer start")
    try:
        quantizer = torch_quantizer(
            quant_mode, model, rand_in, output_dir=quant_dir, device=device)
    except Exception as e:
        print("exception:")
        print(e)
        return
    print("get qunatizer end")

    print("get quantized model start")
    quantized_model = quantizer.quant_model
    print("get quantized model end")

    # evaluate
    output = quantized_model(rand_in)

    # export config
    if quant_mode == 'calib':
        print("export config")
        quantizer.export_quant_config()
        print("export config end")
    # export model
    if quant_mode == 'test':
        print("export xmodel")
        quantizer.export_xmodel(deploy_check=False, output_dir=quant_dir)
        print("export xmodel end")


In [5]:
quantize(float_model=model, 
         input_shape=(1, 1, 28, 28),
         quant_dir='quant_dir_simple',
         quant_mode='calib',
         device=torch.device("cpu"),
         )

before import
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'

[VAIQ_NOTE]: Loading NNDCT kernels...
Before device
get qunatizer start

[VAIQ_NOTE]: Quantization calibration process start up...

[VAIQ_NOTE]: =>Quant Module is in 'cpu'.

[VAIQ_NOTE]: =>Parsing SimpleConvNet...


/opt/vitis_ai/conda/envs/vitis-ai-pytorch/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370120218/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0



[VAIQ_NOTE]: Start to trace model...

[VAIQ_NOTE]: Finish tracing.

[VAIQ_NOTE]: Processing ops...


██████████████████████████████████████████████████| 10/10 [00:00<00:00, 800.00it/s, OpInfo: name = SimpleConvNet/Linear[fc]/216, type = addmm]          


[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(quant_dir_simple/SimpleConvNet.py)
get qunatizer end
get quantized model start

[VAIQ_NOTE]: =>Get module with quantization.
get quantized model end
export config

[VAIQ_NOTE]: =>Exporting quant config.(quant_dir_simple/quant_info.json)
export config end


In [6]:
quantize(float_model=model, 
         input_shape=(1, 1, 28, 28),
         quant_dir='quant_dir_simple',
         quant_mode='test',
         device=torch.device("cpu"),
         )

before import
Before device
get qunatizer start

[VAIQ_NOTE]: Quantization test process start up...

[VAIQ_NOTE]: =>Quant Module is in 'cpu'.

[VAIQ_NOTE]: =>Parsing SimpleConvNet...

[VAIQ_NOTE]: Start to trace model...

[VAIQ_NOTE]: Finish tracing.

[VAIQ_NOTE]: Processing ops...


██████████████████████████████████████████████████| 10/10 [00:00<00:00, 1357.47it/s, OpInfo: name = SimpleConvNet/Linear[fc]/216, type = addmm]         


[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(quant_dir_simple/SimpleConvNet.py)
get qunatizer end
get quantized model start

[VAIQ_NOTE]: =>Get module with quantization.
get quantized model end
export xmodel

[VAIQ_NOTE]: =>Converting to xmodel ...

[VAIQ_NOTE]: =>Successfully convert 'SimpleConvNet' to xmodel.(quant_dir_simple/SimpleConvNet_int.xmodel)
export xmodel end


In [7]:
# compile model
!vai_c_xir --xmodel 'quant_dir_simple/SimpleConvNet_int.xmodel' --arch arch2.json --net_name SimpleConvNet_qu --output_dir build

**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[UNILOG][INFO] Compile mode: dpu
[UNILOG][INFO] Debug mode: function
[UNILOG][INFO] Target architecture: DPUCZDX8G_ISA0_B4096_MAX_BG2
[UNILOG][INFO] Graph name: SimpleConvNet, with op num: 31
[UNILOG][INFO] Begin to compile...
[UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
[UNILOG][INFO] Compile done.
[UNILOG][INFO] The meta json is saved to "/workspace/build/meta.json"
[UNILOG][INFO] The compiled xmodel is saved to "/workspace/build/SimpleConvNet_qu.xmodel"
[UNILOG][INFO] The compiled xmodel's md5sum is 7b14fe95626a1e710caf1d4dfba25b88, and has been saved to "/workspace/build/md5sum.txt"
